In [16]:
import os
import sys
import re

def process_file(input_filename):
    if not input_filename.endswith('.c'):
        raise ValueError("Input file must have a .c extension")

    output_filename = input_filename.replace('.c', '.h')
    header_guard = os.path.basename(output_filename).replace('.', '_').upper()

    with open(input_filename, 'r') as infile:
        lines = infile.readlines()

    defines = []
    current_define = None
    word_count = 0
    in_const_block = False

    output_lines = [f"#ifndef {header_guard}\n", f"#define {header_guard}\n\n"]

    for line in lines:
        line = line.strip()

        # 处理包含const的行
        if in_const_block == True or line.startswith("const"):
            in_const_block = True
        else:
            continue
        
        if line.startswith('#define'):
            if current_define is not None:
                defines.append((current_define, word_count))
            current_define = line
            word_count = 0
        elif current_define and not line.startswith('#'):  # Count numbers between #define sections
            if re.match(r"I16\(\d+,\s*\d+\)", line):
                word_count += len(re.findall(r"I16\(\d+,\s*\d+\)", line))
            else:
                word_count += len(re.findall(r'\b\d+\b', line))

    # Add the last define if applicable
    if current_define is not None:
        defines.append((current_define, word_count))

    # Process the defines with their offsets
    offset = 0
    for define, count in defines:
        output_lines.append(f"{define} {offset}\n")
        offset += count

    output_lines.append("\n#endif\n")

    with open(output_filename, 'w') as outfile:
        outfile.writelines(output_lines)

    print(f"Header file '{output_filename}' generated successfully.")

if __name__ == "__main__":
    try:
        process_file("consts.c")
    except Exception as e:
        print(f"Error: {e}")
        sys.exit(1)


Header file 'consts.h' generated successfully.
